**Tratamento de Dados - Planilha Controle - Frota de Veículos**

Organizar e padronizar as informações da planilha Controle, de forma que ela sirva como base central de gestão de veículos.

O objetivo é integrar e consolidar dados da planilha Central de Veículos, garantindo maior rastreabilidade, consistência e facilidade de atualização das informações.

Além disso, será criada uma planilha de Histórico, permitindo registrar alterações, movimentações e eventos relevantes de cada veículo ao longo do tempo — aprimorando o acompanhamento e o controle da base de dados.

In [194]:
import pandas as pd 
import numpy as np



In [195]:

# Caminho do arquivo
CAMINHO_ARQUIVO = r'C:\Users\LENOVO\Desktop\PORTIFOLIO\CASE FROTA\04112025 - Planilha Controle.xlsx'
ABA_CONTROLE = 'Controle'

# === 1 Carregar dados ===
carregar_dados_controle = pd.read_excel(CAMINHO_ARQUIVO, sheet_name=ABA_CONTROLE)

# === 2 Remover linhas totalmente vazias ===
carregar_dados_controle.dropna(how='all', inplace=True)

# === 3 Selecionar apenas as colunas relevantes ===
colunas_relevantes = [
    'PLACA', 'RENAVAN', 'A/M', 'MARCA', 'COR', 'LOCADO',
    'Unnamed: 7', 'RESERVA', 'CONTATO'
]
carregar_dados_controle = carregar_dados_controle[colunas_relevantes]

# === 4 Renomear colunas para nomes padronizados ===
novos_nomes = {
    'MARCA': 'MODELO',
    'LOCADO': 'CONTRATO',
    'Unnamed: 7': 'LOCADO (Por quem?)',
    'RESERVA': 'PLACA RESERVADA',
    'CONTATO': 'HISTORICO'
}
carregar_dados_controle.rename(columns=novos_nomes, inplace=True)

# === 5 Criar novas colunas vazias ===
novas_colunas = [
    'LOCADO (SIM OU NÃO)', 'STATUS CARRO', 'RESERVA PELO CLIENTE (SIM OU NÃO)',
    'CARRO SUBSTITUTO', 'CARRO SUBSTITUTO (PLACA)', 'CONTATO', 'OBSERVAÇÃO'
]
for coluna in novas_colunas:
    carregar_dados_controle[coluna] = np.nan  # ou '' se preferir vazio em texto

# === 6 Reorganizar a ordem final das colunas ===
ordem_colunas = [
    'PLACA', 'RENAVAN', 'A/M', 'MODELO', 'COR', 'LOCADO (SIM OU NÃO)',
    'CONTRATO', 'LOCADO (Por quem?)', 'STATUS CARRO',
    'RESERVA PELO CLIENTE (SIM OU NÃO)', 'PLACA RESERVADA',
    'CARRO SUBSTITUTO', 'CARRO SUBSTITUTO (PLACA)',
    'HISTORICO', 'CONTATO', 'OBSERVAÇÃO'
]
carregar_dados_controle = carregar_dados_controle[ordem_colunas]

# === 7 Resumo de valores nulos ===
nulos_por_coluna = carregar_dados_controle.isnull().sum()
print("Contagem de valores nulos por coluna:\n")
print(nulos_por_coluna)



Contagem de valores nulos por coluna:

PLACA                                  3
RENAVAN                                9
A/M                                   18
MODELO                                 3
COR                                   12
LOCADO (SIM OU NÃO)                  230
CONTRATO                               5
LOCADO (Por quem?)                   194
STATUS CARRO                         230
RESERVA PELO CLIENTE (SIM OU NÃO)    230
PLACA RESERVADA                      186
CARRO SUBSTITUTO                     230
CARRO SUBSTITUTO (PLACA)             230
HISTORICO                             28
CONTATO                              230
OBSERVAÇÃO                           230
dtype: int64


In [196]:
# === 8 Limpeza de linhas com placas nulas ===

carregar_dados_controle = carregar_dados_controle[carregar_dados_controle['PLACA'].notna()]
carregar_dados_controle = carregar_dados_controle[carregar_dados_controle['PLACA'].str.strip() != '']


print("Linhas restantes após exclusão:", len(carregar_dados_controle))


Linhas restantes após exclusão: 227


In [197]:

# === 9 Padronizar valores na coluna 'COR' ===

# Remove espaços extras e converte tudo para maiúsculas
carregar_dados_controle['COR'] = carregar_dados_controle['COR'].astype(str).str.strip().str.upper()

# Dicionário de substituições
cor_substituicoes = {
    'PRETO': 'PRETA',
    'PRETO ': 'PRETA',   # caso tenha espaço no final
    'CINZA ': 'CINZA',   # corrige espaço extra
    'INCENDIO': '',       # limpa valores incorretos
    'ROUBADO': ''         # idem
}

# Aplicar substituições de forma mais limpa
carregar_dados_controle['COR'] = carregar_dados_controle['COR'].replace(cor_substituicoes)

# === 10 Exibir resultados de verificação ===
print("\nValores únicos da coluna 'COR':")
print(carregar_dados_controle['COR'].unique())

print("\nContagem de valores nulos por coluna:")
print(carregar_dados_controle.isnull().sum())




Valores únicos da coluna 'COR':
['PRETA' 'CINZA' 'PRATA' 'AZUL' 'BRANCA' 'NAN' '']

Contagem de valores nulos por coluna:
PLACA                                  0
RENAVAN                                6
A/M                                   15
MODELO                                 0
COR                                    0
LOCADO (SIM OU NÃO)                  227
CONTRATO                               2
LOCADO (Por quem?)                   193
STATUS CARRO                         227
RESERVA PELO CLIENTE (SIM OU NÃO)    227
PLACA RESERVADA                      183
CARRO SUBSTITUTO                     227
CARRO SUBSTITUTO (PLACA)             227
HISTORICO                             26
CONTATO                              227
OBSERVAÇÃO                           227
dtype: int64


In [198]:
# === 11 Exibir valores de CONTRATO === Regras de Negócio 

carregar_dados_controle['LOCADO (SIM OU NÃO)'] = 'NÃO'

carregar_dados_controle['CONTRATO'] = (
    carregar_dados_controle['CONTRATO']
    .astype(str)
    .str.strip()
    .str.upper()
)

carregar_dados_controle.loc[
    carregar_dados_controle['CONTRATO'].str.contains('SEAP', na=False),
    'LOCADO (SIM OU NÃO)'
] = 'SIM'

carregar_dados_controle.loc[
    carregar_dados_controle['CONTRATO'].str.contains('ZADAR', na=False),
    'LOCADO (SIM OU NÃO)'
] = 'SIM'

carregar_dados_controle.loc[
    carregar_dados_controle['CONTRATO'].str.contains('REI', na=False),
    'LOCADO (SIM OU NÃO)'
] = 'SIM'

carregar_dados_controle.loc[
    carregar_dados_controle['CONTRATO'].str.contains('GSI', na=False),
    'LOCADO (SIM OU NÃO)'
] = 'SIM'

carregar_dados_controle.loc[
    carregar_dados_controle['CONTRATO'].str.contains('NEW FAST', na=False),
    'LOCADO (SIM OU NÃO)'
] = 'SIM'

carregar_dados_controle.loc[
    carregar_dados_controle['CONTRATO'].str.contains('MG', na=False),
    'LOCADO (SIM OU NÃO)'
] = 'SIM'

carregar_dados_controle.loc[
    carregar_dados_controle['CONTRATO'].str.contains('PM', na=False),
    'LOCADO (SIM OU NÃO)'
] = 'Indisponível'

# === 12 Exibir Resultado do Tratamento 

filtro_seap = carregar_dados_controle['CONTRATO'].str.contains('SEAP', case=False, na=False)

print(carregar_dados_controle.loc[filtro_seap, ['CONTRATO', 'LOCADO (SIM OU NÃO)']])

filtro_rei = carregar_dados_controle['CONTRATO'].str.contains('REI', case=False, na=False)

print(carregar_dados_controle.loc[filtro_rei, ['CONTRATO', 'LOCADO (SIM OU NÃO)']])

filtro_zadar = carregar_dados_controle['CONTRATO'].str.contains('ZADAR', case=False, na=False)

print(carregar_dados_controle.loc[filtro_zadar, ['CONTRATO', 'LOCADO (SIM OU NÃO)']])

filtro_gsi = carregar_dados_controle['CONTRATO'].str.contains('GSI', case=False, na=False)

print(carregar_dados_controle.loc[filtro_gsi, ['CONTRATO', 'LOCADO (SIM OU NÃO)']])

filtro_newfast = carregar_dados_controle['CONTRATO'].str.contains('NEW FAST', case=False, na=False)

print(carregar_dados_controle.loc[filtro_newfast, ['CONTRATO', 'LOCADO (SIM OU NÃO)']])

filtro_mg = carregar_dados_controle['CONTRATO'].str.contains('MG', case=False, na=False)

print(carregar_dados_controle.loc[filtro_mg, ['CONTRATO', 'LOCADO (SIM OU NÃO)']])

filtro_pm = carregar_dados_controle['CONTRATO'].str.contains('PM', case=False, na=False)

print(carregar_dados_controle.loc[filtro_pm, ['CONTRATO', 'LOCADO (SIM OU NÃO)']])




       CONTRATO LOCADO (SIM OU NÃO)
0          SEAP                 SIM
2          SEAP                 SIM
3          SEAP                 SIM
4          SEAP                 SIM
5          SEAP                 SIM
..          ...                 ...
72         SEAP                 SIM
73         SEAP                 SIM
75         SEAP                 SIM
79  THIAGO SEAP                 SIM
80   CARMO SEAP                 SIM

[74 rows x 2 columns]
                 CONTRATO LOCADO (SIM OU NÃO)
1                     REI                 SIM
23                    REI                 SIM
30                    REI                 SIM
97                    REI                 SIM
243             REI-BISPO                 SIM
260                   REI                 SIM
261                   REI                 SIM
263                   REI                 SIM
265                   REI                 SIM
266                   REI                 SIM
267                   REI              

In [204]:

# === 13 Tratamento Regra de Negócio - Tratamento do Status no Campo Correto

carregar_dados_controle['STATUS CARRO'] = 'OK'

filtro_roubado = carregar_dados_controle['CONTRATO'].str.contains('ROUBADO', na=False)
filtro_roubada = carregar_dados_controle['CONTRATO'].str.contains('ROUBADA', na=False)
filtro_vendida = carregar_dados_controle['CONTRATO'].str.contains('VENDIDA', na=False)
filtro_vendido = carregar_dados_controle['CONTRATO'].str.contains('VENDIDO', na=False)
filtro_percatotal = carregar_dados_controle['CONTRATO'].str.contains('PERCA TOTAL', na=False)


carregar_dados_controle.loc[filtro_roubado, 'STATUS CARRO'] = 'ROUBADO'
carregar_dados_controle.loc[filtro_roubado, 'CONTRATO'] = ''

carregar_dados_controle.loc[filtro_roubada, 'STATUS CARRO'] = 'ROUBADA'
carregar_dados_controle.loc[filtro_roubada, 'CONTRATO'] = ''

carregar_dados_controle.loc[filtro_vendida, 'STATUS CARRO'] = 'VENDIDA'
carregar_dados_controle.loc[filtro_vendida, 'CONTRATO'] = ''

carregar_dados_controle.loc[filtro_vendido, 'STATUS CARRO'] = 'VENDIDO'
carregar_dados_controle.loc[filtro_vendido, 'CONTRATO'] = ''

carregar_dados_controle.loc[filtro_percatotal, 'STATUS CARRO'] = 'PERCA TOTAL'
carregar_dados_controle.loc[filtro_percatotal, 'CONTRATO'] = ''

carregar_dados_controle

,PLACA,RENAVAN,A/M,MODELO,COR,LOCADO (SIM OU NÃO),CONTRATO,LOCADO (Por quem?),STATUS CARRO,RESERVA PELO CLIENTE (SIM OU NÃO),PLACA RESERVADA,CARRO SUBSTITUTO,CARRO SUBSTITUTO (PLACA),HISTORICO,CONTATO,OBSERVAÇÃO
0,SRW 0E11,1.401714e+09,24/24,HILUX SW4,PRETA,SIM,SEAP,NaN,OK,NaN,NaN,NaN,NaN,Data Entrega SEAP 08/09/2024 - Trocou pela RJM...,NaN,NaN
1,TTF 4J91,1.427801e+09,25/26,COROLLA CROSS,PRETA,SIM,REI,NaN,OK,NaN,TUB 1B93,NaN,NaN,Data Entega SEAP 08/05/2025 - Foi para SP pela...,NaN,NaN
2,RIU 8I21,1.341462e+09,23/23,COROLLA CROSS,PRETA,SIM,SEAP,NaN,OK,NaN,NaN,NaN,NaN,Data Entrega SEAP 13/06/2024 = Entregar a Zada...,NaN,NaN
3,TTD 0G45,1.419546e+09,24/25,COROLLA CROSS,PRETA,SIM,SEAP,NaN,OK,NaN,TTC 3J31,NaN,NaN,Data Entrega SEAP 28/01/2025 - Antonio (Dr. Ad...,NaN,NaN
4,RJQ 3G73,1.264480e+09,21/22,COROLLA XEI,PRETA,SIM,SEAP,NaN,OK,NaN,RKT 4D19,NaN,NaN,Data Entrega SEAP 07/08/2023-,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,RKN 5F67,1.289651e+09,21/22,COROLLA XEI,PRATA,NÃO,,NaN,OK,NaN,NaN,NaN,NaN,Foi dia MG3 p/Blindagem=Veio da Blindagem dia ...,NaN,NaN
342,TTB 4E37,1.435022e+09,25/25,COROLLA XEI,PRETA,NÃO,,Sinistro do Bispo,OK,NaN,NaN,NaN,NaN,Data Entrega SEAP 15/05/2025 - Data Sinistro ...,NaN,NaN
343,RJX 6G74,1.317619e+09,22/23,COROLLA XEI,PRETA,NÃO,,NaN,OK,NaN,NaN,NaN,NaN,Entregue p/Blindagem dia 15/08/22 para Forc Ca...,NaN,NaN
344,RKE 1D70,1.239449e+09,20/21,COROLLA,PRETA,NÃO,,NaN,OK,NaN,LUI 7F61,NaN,NaN,Estava emprestado com o Dr. João dia 08 a 17/0...,NaN,NaN


In [205]:
# Caminho onde você quer salvar o novo arquivo
caminho_saida = r'C:\Users\LENOVO\Desktop\PORTIFOLIO\CASE FROTA\saida.xlsx'

# Exportar para Excel
carregar_dados_controle.to_excel(caminho_saida, index=False)

print("✅ Arquivo Excel gerado com sucesso em:")
print(caminho_saida)

✅ Arquivo Excel gerado com sucesso em:
C:\Users\LENOVO\Desktop\PORTIFOLIO\CASE FROTA\saida.xlsx
